In [3]:
#예제 1
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (55, 'Brazil'),
    (7, 'Russia')
]
country_code = {country: code for code, country in DIAL_CODES}
country_code



{'China': 86, 'India': 91, 'United States': 1, 'Brazil': 55, 'Russia': 7}

In [4]:
{code: country.upper() for country, code in country_code.items() if code < 66}

{1: 'UNITED STATES', 55: 'BRAZIL', 7: 'RUSSIA'}

In [8]:
from os import WCOREDUMP
#예제 2
"""단어가 나타나는 위치를 가리키는 인덱스를 만든다."""

import sys, re

WORD_RE = re.compile(r'\w+')
index = {}
with open("sejong.txt", encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            occurance = index.get(word, [])
            occurance.append(location)
            index[word] = occurance

for word in sorted(index, key=str.upper):
    print(word, index[word])

1 [(3, 447)]
1397 [(1, 39)]
1418 [(3, 118)]
1422 [(3, 441)]
1439 [(7, 6)]
1450 [(1, 55), (7, 123)]
15 [(1, 32)]
2 [(5, 168)]
3 [(5, 171)]
30 [(1, 46)]
4 [(5, 479)]
5 [(5, 482)]
6 [(7, 39)]
a [(3, 184), (5, 339)]
abdicated [(3, 224)]
absolute [(3, 359)]
acted [(7, 85)]
advancements [(5, 185)]
alphabet [(1, 316), (5, 158)]
also [(5, 431)]
amendments [(5, 88)]
and [(1, 260), (3, 94), (3, 378), (3, 398), (5, 36), (5, 64), (5, 131), (5, 209), (5, 225), (5, 323), (7, 42)]
areas [(5, 421)]
as [(1, 107), (1, 211), (1, 278), (3, 167), (7, 91)]
became [(7, 15)]
brother [(3, 151)]
campaigns [(5, 300)]
Chungnyeong [(3, 31)]
commonly [(1, 92)]
Confucian [(5, 26), (5, 44)]
conquered [(5, 411)]
continued [(3, 402)]
created [(5, 123)]
crown [(3, 170)]
Crown [(7, 62)]
death [(3, 432)]
died [(7, 109)]
dynasty [(1, 178)]
early [(3, 271)]
economic [(5, 262)]
eldest [(3, 144), (7, 50)]
Emeritus [(3, 307)]
enacted [(5, 68)]
encouraged [(5, 174)]
establishing [(5, 373)]
executive [(3, 368)]
favor [(3, 257)]


In [9]:
#예제 3
import sys, re

WORD_RE = re.compile(r'\w+')
index = {}

with open("sejong.txt", encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
      for match in WORD_RE.finditer(line):
        word = match.group()
        column_no = match.start() + 1
        location = (line_no, column_no)
        index.setdefault(word, []).append(location)
for word in sorted(index, key=str.upper):
    print(word, index[word])

1 [(3, 447)]
1397 [(1, 39)]
1418 [(3, 118)]
1422 [(3, 441)]
1439 [(7, 6)]
1450 [(1, 55), (7, 123)]
15 [(1, 32)]
2 [(5, 168)]
3 [(5, 171)]
30 [(1, 46)]
4 [(5, 479)]
5 [(5, 482)]
6 [(7, 39)]
a [(3, 184), (5, 339)]
abdicated [(3, 224)]
absolute [(3, 359)]
acted [(7, 85)]
advancements [(5, 185)]
alphabet [(1, 316), (5, 158)]
also [(5, 431)]
amendments [(5, 88)]
and [(1, 260), (3, 94), (3, 378), (3, 398), (5, 36), (5, 64), (5, 131), (5, 209), (5, 225), (5, 323), (7, 42)]
areas [(5, 421)]
as [(1, 107), (1, 211), (1, 278), (3, 167), (7, 91)]
became [(7, 15)]
brother [(3, 151)]
campaigns [(5, 300)]
Chungnyeong [(3, 31)]
commonly [(1, 92)]
Confucian [(5, 26), (5, 44)]
conquered [(5, 411)]
continued [(3, 402)]
created [(5, 123)]
crown [(3, 170)]
Crown [(7, 62)]
death [(3, 432)]
died [(7, 109)]
dynasty [(1, 178)]
early [(3, 271)]
economic [(5, 262)]
eldest [(3, 144), (7, 50)]
Emeritus [(3, 307)]
enacted [(5, 68)]
encouraged [(5, 174)]
establishing [(5, 373)]
executive [(3, 368)]
favor [(3, 257)]


In [10]:
#예제 4
import sys, re, collections

WORD_RE = re.compile(r'\w+')

index = collections.defaultdict(list) #defaultdict!

with open("sejong.txt", encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
      for match in WORD_RE.finditer(line):
        word = match.group()
        column_no = match.start() + 1
        location = (line_no, column_no)
        index[word].append(location)
for word in sorted(index, key=str.upper):
    print(word, index[word])

1 [(3, 447)]
1397 [(1, 39)]
1418 [(3, 118)]
1422 [(3, 441)]
1439 [(7, 6)]
1450 [(1, 55), (7, 123)]
15 [(1, 32)]
2 [(5, 168)]
3 [(5, 171)]
30 [(1, 46)]
4 [(5, 479)]
5 [(5, 482)]
6 [(7, 39)]
a [(3, 184), (5, 339)]
abdicated [(3, 224)]
absolute [(3, 359)]
acted [(7, 85)]
advancements [(5, 185)]
alphabet [(1, 316), (5, 158)]
also [(5, 431)]
amendments [(5, 88)]
and [(1, 260), (3, 94), (3, 378), (3, 398), (5, 36), (5, 64), (5, 131), (5, 209), (5, 225), (5, 323), (7, 42)]
areas [(5, 421)]
as [(1, 107), (1, 211), (1, 278), (3, 167), (7, 91)]
became [(7, 15)]
brother [(3, 151)]
campaigns [(5, 300)]
Chungnyeong [(3, 31)]
commonly [(1, 92)]
Confucian [(5, 26), (5, 44)]
conquered [(5, 411)]
continued [(3, 402)]
created [(5, 123)]
crown [(3, 170)]
Crown [(7, 62)]
death [(3, 432)]
died [(7, 109)]
dynasty [(1, 178)]
early [(3, 271)]
economic [(5, 262)]
eldest [(3, 144), (7, 50)]
Emeritus [(3, 307)]
enacted [(5, 68)]
encouraged [(5, 174)]
establishing [(5, 373)]
executive [(3, 368)]
favor [(3, 257)]


In [11]:
#예제 5
class StrKeyDict0(dict):
  # dict 상속
    def __missing__(self, key):
      """
        instance가 없으면 __ㅡmissing__() 메서드를 재귀적으로 호출
      """
      if isinstance(key, str):
          #키가 문자열인지 확인. 존재하지 않으면 에러  KeyErorr 발생
          raise KeyError(key)
      return self[str(key)] #키에서 문자열 만들고 조회
    def get(self, key, default=None):
      """
        get()메서드는 self[key] 표기법을 이용, __getitem()__메서드에 위임
        키가 없으면 __missing()__ 메서드가 작동할 수 있게 함
      """
      try:
          return self[key]
      except KeyError:
        #KeyErorr가 발생하면 __missing()__ 메서드 실패 -> default 반환
          return default
    def __contains__(self, key):
      # 'key in self': 재귀적으로 __contains()__ 호출
      # 수정하지 않은 키(문자열이 아닐수 있는)를 검색 후, 키에서 만든 문자열 검색
      return key in self.keys() or str(key) in self.keys()

In [12]:
#예제 6
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']

'two'

In [13]:
d[4]

'four'

In [14]:
d.get('2')

'two'

In [15]:
d.get('4')

'four'

In [16]:
d.get(1,'N/A')

'N/A'

In [17]:
2 in d

True

In [18]:
#예제 7
d[1]

KeyError: '1'

In [20]:
#예제 8
class StrKeyDict(collections.UserDict):

  def __missing__(self, key):
    if isinstance(key, str):
      raise KeyError(key)
    return self[str(key)]

  def __contains__(self, key):
    return str(key) in self.data

  def __setitem__(self, key, item):
    self.data[str(key)] = item

In [21]:
#예제 9
from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [22]:
d_proxy[1]

'A'

In [23]:
d_proxy[2] = 'x'

TypeError: 'mappingproxy' object does not support item assignment

In [24]:
d[2] = 'B'
d_proxy

mappingproxy({1: 'A', 2: 'B'})

In [25]:
#예제 10
l = ["spam", "spam", "eggs", "spam"]
set(l)

{'eggs', 'spam'}

In [26]:
list(set(l))

['eggs', 'spam']

In [27]:
#예제 11
l = set(["spam", "spam", "eggs", "spam", "eggs", "meats", "onions", "onions"])
m = set(["eggs", "onions"])

found = len(l & m)
found

2

In [28]:
found = 0
for n in m:
  if n in l:
    found += 1
found

2

In [29]:
found = len(l.intersection(m))
found

2

In [30]:
#예제 12
s = {1}
print(type(s))
print(s)

s.pop()
print(s)

<class 'set'>
{1}
set()


In [31]:
#예제 13
from unicodedata import name
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

In [33]:
#예제 14
DIAL_CODES = [
    (82, 'Korea'),
    (91, 'India'),
    (1, 'United States'),
    (92, 'Pakistan'),
    (7, 'Russia')
]

#인구가 많은 순서대로 정렬된 튜플 생성
d1 = dict(DIAL_CODES)
print(f'd1: {d1.keys()}')
#국제전화 코드로 정렬된 튜플 생성
d2 = dict(sorted(DIAL_CODES))
print(f'd2: {d2.keys()}')
#국가명으로 정렬된 튜플 생성
d3 = dict(sorted(DIAL_CODES, key=lambda x:x[1]))
print(f'd3: {d3.keys()}')

assert d1 == d2 and d2 == d3
#딕셔너리가 모두 동일한 키-값 쌍을 갖고 있음을 판단

d1: dict_keys([82, 91, 1, 92, 7])
d2: dict_keys([1, 7, 82, 91, 92])
d3: dict_keys([91, 82, 92, 7, 1])
